In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/credit_card_fraud.csv')
print(df.shape)
df.head()

(8000, 20)


,Transaction Date and Time,Transaction Amount,Cardholder Name,Card Number (Hashed or Encrypted),Merchant Name,Merchant Category Code (MCC),Transaction Location (City or ZIP Code),Transaction Currency,Card Type,Card Expiration Date,CVV Code (Hashed or Encrypted),Transaction Response Code,Transaction ID,Fraud Flag or Label,Previous Transactions,Transaction Source,IP Address,Device Information,User Account Information,Transaction Notes
0,2022-09-24 13:54:27,285.88,Shray Soman,daca51bffe0fc4eaaa7c430917f94b75fd893712492254...,"Rajagopalan, Ghose and Kant",3590,Khammam,INR,MasterCard,04/29,01299ac65733b5a3d774265fbfe8396b8611e5e3321855...,5,b7f69cbc-a03d-41f8-adca-75920b0242c3,1,None,Online,18.106.240.6,Tablet,NaN,Consequatur corporis minima ad vero deserunt n...
1,2020-07-24 11:20:13,1777.32,Lakshit Bakshi,d5366dd9be3a0266c0252baceaa6332210ed4dbd0ef252...,Sule PLC,7277,Vasai-Virar,EUR,American Express,03/26,ad8fa913d25b3970c6efa8ca504da8ba670ce2a9cf012d...,12,ad53cc8e-8412-422e-8cad-4176daac8387,1,None,Online,212.48.185.128,Mobile,jloyal,Dolore repellendus odio deleniti. Eaque quibus...
2,2023-03-18 01:05:36,3939.01,Riya Bobal,4737a6384aa1c3c31f7768b86633d6a8401dd6ab4a8ff1...,Badal PLC,9297,Nangloi Jat,USD,Visa,11/29,eb3be230bbd2844b1f5d8f2e4fab9ffba8ab22cfeeb69c...,5,f09cecd6-0d74-4551-8089-a31d64e6b9c5,1,3 or more,In-Person,17.190.112.46,Mobile,NaN,Unde beatae perspiciatis sapiente. Voluptates ...
3,2021-01-07 21:53:04,376.44,Mohanlal Balakrishnan,4721806eed8f2663bb597ff13e79a294de318358fbb54b...,Konda-Sodhi,5686,Ramagundam,USD,Visa,09/25,891d46993a36d78392247c642138cede01d9841daab1d9...,5,b208ae0b-4c4f-428f-b6b9-5360b288b947,0,2,In-Person,153.136.24.104,Tablet,NaN,Cupiditate repellendus necessitatibus quo occa...
4,2021-12-16 06:22:24,1687.33,Mannat Rout,9deacc3a9efd6e382826d400620aca5f23ed94327578e5...,Dua Ltd,2940,Adoni,INR,MasterCard,03/31,48a1a756f2d83f1dc57bbf14052b70a6f40d0fceed6662...,5,6b4e4e43-5b73-4906-9973-299a1b2a5e71,1,2,Online,196.153.28.131,Desktop,NaN,Molestias assumenda consectetur itaque veritat...


In [3]:
# Dropping columns we won't use as they are either obfuscated or of little interest.
df.drop(
    columns=[
        'Cardholder Name',
        'Card Number (Hashed or Encrypted)',
        'CVV Code (Hashed or Encrypted)',
        'Transaction ID',
        'IP Address',
        'Transaction Notes',
        'User Account Information' ],
    inplace=True,
    errors='ingore'
)

# Giving columns more concise names, allowing us to access via .notation
df.rename(
    columns={
        'Transaction Date and Time'                 : 'datetime',
        'Transaction Amount'                        : 'amount',
#        'Cardholder Name'                           : 'cardholder',
        'Merchant Name'                             : 'merchant',
        'Merchant Category Code (MCC)'              : 'mcc',
        'Transaction Location (City or ZIP Code)'   : 'location',
        'Card Expiration Date'                      : 'exp_date',
        'Transaction Currency'                      : 'currency',
        'Card Type'                                 : 'card_type',
        'Card Expiration Date'                      : 'card_exp_date',
        'Transaction Response Code'                 : 'response_code',
        'Fraud Flag or Label'                       : 'flagged_as_fraud',
        'Previous Transactions'                     : 'previous_transactions',
        'Transaction Source'                        : 'source',
        'Device Information'                        : 'device',
#        'User Account Information'                  : 'user_account'
    }, inplace= True
)
df.dtypes

datetime                  object
amount                   float64
merchant                  object
mcc                        int64
location                  object
currency                  object
card_type                 object
card_exp_date             object
response_code              int64
flagged_as_fraud           int64
previous_transactions     object
source                    object
device                    object
dtype: object

# Issues:
| Column                | Notes                        |
| --------------------- | ---------------------------- |
| datetime              | change to datetime           |
| amount                |                              |
| merchant              |                              |
| mcc                   | codes, but no human meaning? |
| location              | India city names?            |
| currency              |                              |
| card_type             |                              |
| card_exp_date         | change to date type          |
| response_code         | What are these?              |
| flagged_as_fraud      | Should be boolean            |
| previous_transactions |                              |
| source                |                              |
| device                |                              |

In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])

df.describe()

,amount,mcc,response_code,flagged_as_fraud
count,8000.000000,8000.000000,8000.000000,8000.000000
mean,2496.356036,5484.150375,5.637500,0.498625
std,1451.221326,2608.164617,4.928147,0.500029
min,1.090000,1000.000000,0.000000,0.000000
25%,1242.580000,3230.750000,0.000000,0.000000
50%,2492.460000,5455.000000,5.000000,0.000000
75%,3739.522500,7761.000000,12.000000,1.000000
max,4996.700000,9999.000000,12.000000,1.000000


In [5]:
df['location'].value_counts()

Ghaziabad     53
Aurangabad    51
Darbhanga     39
Bettiah       38
Malda         37
              ..
Hosur         15
Kumbakonam    15
Sagar         15
Rampur        13
Jaunpur       11
Name: location, Length: 316, dtype: int64

In [6]:
df['flagged_as_fraud'].value_counts()
# Good distribution for machine learning

0    4011
1    3989
Name: flagged_as_fraud, dtype: int64